In [2]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [4]:
data_path = "train_single_smaller.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 2113
First example:
{'role': 'system', 'content': '당신은 유능한 순위예측 전문가입니다.'}
{'role': 'user', 'content': '선수1: 등급: A1, 성별: 남, 나이: 44.0, 체중: 56.0, 최근6회차_평균착순점: 5.85, 최근6회차_평균득점: 3.38, 최근6회차_승률: 15.4, 최근6회차_연대율2: 38.5, 최근6회차_연대율3: 76.9, 최근6회차_평균ST: 0.17, 연간성적_평균착순점: 6.83, 연간성적_연대율: 63.0, FL: F2L0, 평균사고점: 3.1, 출주횟수: 17.0, 모터_평균착순점: 5.99, 모터_연대율2: 40.8, 모터_연대율3: 59.2, 보트_평균착순점: 5.42, 보트_연대율: 36.7, 1코스_성적: 62.87, 2코스_성적: 47.91, 3코스_성적: 32.69, 4코스_성적: 31.80, 5코스_성적: 36.78, 6코스_성적: 19.06, 최근1경기_착순: -1, 최근2경기_착순: 3, 최근3경기_착순: 3, 최근4경기_착순: -1, 최근5경기_착순: 2, 최근6경기_착순: 1, 최근7경기_착순: 2, 최근8경기_착순: 3 선수2: 등급: A1, 성별: 남, 나이: 49.0, 체중: 57.0, 최근6회차_평균착순점: 4.61, 최근6회차_평균득점: 4.56, 최근6회차_승률: 5.6, 최근6회차_연대율2: 22.2, 최근6회차_연대율3: 44.4, 최근6회차_평균ST: 0.26, 연간성적_평균착순점: 5.79, 연간성적_연대율: 45.7, FL: F0L0, 평균사고점: 0.03, 출주횟수: 36.0, 모터_평균착순점: 5.73, 모터_연대율2: 40.3, 모터_연대율3: 56.1, 보트_평균착순점: 6.08, 보트_연대율: 45.9, 1코스_성적: 56.46, 2코스_성적: 56.58, 3코스_성적: 66.32, 4코스_성적: 32.72, 5코스_성적: 2.58, 6코스_성적: 14.32, 최근1경기_착순: -1, 최근2

In [5]:
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [6]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [8]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 3256, 3262
mean / median: 3261.156649313772, 3261.0
p5 / p95: 3260.0, 3262.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 17, 17
mean / median: 17.0, 17.0
p5 / p95: 17.0, 17.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning
